In [46]:
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

import nltk
nltk.download("popular") # required to download the stopwords lists

from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')
count_vectorizer = CountVectorizer(stop_words = spanish_stopwords+english_stopwords)

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [2]:
mainpath = "C:/Users/guill/Documents/Universidad/PlataformaRefugiados/NAUTIA/DesarrolloPy/DataSetOriginales"

def dfFix(df,col1 = False,col2 = False):
    result = df.copy()
    if(col1):
        x = result.columns.get_loc(col1)
        result.drop(result.columns[0:x],axis = 1, inplace = True)
    if(col2):
        y = result.columns.get_loc(col2)
        result.drop(result.columns[y:],axis = 1, inplace = True)
    return result

def concatDF(df1,df2):
    return  pd.concat([df1,df2],axis = 1, ignore_index = True, sort = True)

def dropRow(df,index):
    return df.drop(df.index[index])

def mkCSV(df,fileName):
    df.to_csv('DataSetFinales/'+fileName,header = False, index=False) #Header e index a false para no mostrarlo en el csv
    
def getPath(mainpath,filename):
    return os.path.join(mainpath, filename)

def fixBibliography(df):
    df = dfFix(df,"GENERAL INFORMATION - COUNTRY LEVEL")
    df.columns = ['GeneralInfo', 'CommunityCountry', 'RefugeeCountry']
    df.set_index('GeneralInfo', inplace = True)
    df = df.transpose()
    return df

def getSubColumnNames(df,x):
    columns = df.columns
    array = []
    for column in columns:
        column = column[x:]
        array.append(column)
    return pd.DataFrame(array) 

def addInstitutionAndType(df,array1,array2,instType,index):
    refugees = dropRow(df,index)
    refugees = refugees.dropna(axis = 1)
    refugees = np.array(refugees)
    for row in refugees:
        for elem in row:
            array1 = np.append(array1,elem)
            array2 = np.append(array2,instType)
    return array1,array2

def politicalActor(df1,df2,df3,df4,df5,index):
    institution = []
    instType = []

    institution, instType  = addInstitutionAndType(df1,institution,instType,'Public Institution',index)
    institution, instType  = addInstitutionAndType(df2,institution,instType,'Private Institution',index)
    institution, instType  = addInstitutionAndType(df3,institution,instType,'NGO',index)
    institution, instType  = addInstitutionAndType(df4,institution,instType,'International Agency',index)
    institution, instType  = addInstitutionAndType(df5,institution,instType,'Local',index)

    institution = pd.DataFrame(institution)
    institution = institution.reset_index(drop = True)
    instType = pd.DataFrame(instType)
    instType = instType.reset_index(drop = True)
    
    return concatDF(institution,instType)

def separateValues(df):
    array = np.array(df)
    corpus = []
    for row in array:
        for elem in row:
            corpus = np.append(corpus,[elem])
    X = count_vectorizer.fit_transform(corpus)
    array = count_vectorizer.get_feature_names()
    return pd.DataFrame(array)  

In [3]:
pd.options.display.max_columns = None

In [4]:
Bibliography = pd.read_excel(getPath(mainpath,"Bibliography_120220.xlsx"))
Bibliography = fixBibliography(Bibliography)
Entities = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Entities_Interview_results.csv"))
LocalLeaders = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Local_leaders_v3_results.csv"))
HouseHold = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Survey_household_v6_results.csv"))
WomenGroup = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Women_Focus_Group2_results.csv"))
SanitationInfra = pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv"))
Priorities = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Priorities_v3_results.csv"))
GeneralForm = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_form_v3_results.csv"))
PublicSpace = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Public_Space_results.csv"))
WaterInf = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Water_Infrastructure_results.csv"))
SanitationInf = pd.read_csv(getPath(mainpath,"NAUTIA_V1_0_Sanitation_Infrastructre_results.csv"))
WasteManagementInf = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Waste_Management_Infrastructure_results.csv"))
EnergyINF = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Energy_Infrastructure_results.csv"))
Business = pd.read_csv(getPath(mainpath,"NAUTIA1_0_Business_surveys_v3_results.csv"))
MobilityINF = pd.read_csv(getPath(mainpath,"NAUTIA_1_0__Transport_servicesaccess_points_results.csv")) 
ComunalServices = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Communal_Services_results.csv"))
GeneralCitizen = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_General_Citizen_Focus_Group_results.csv"))
Shelter = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Shelter_results.csv"))
FarmyardCrop = pd.read_csv(getPath(mainpath,"NAUTIA_1_0_Farmyard_and_Crops_results.csv"))

In [5]:
Bibliography.shape

(2, 191)

In [47]:
Entities

,start,end,Type_of_settlement,Type_Host_Community,Identification:Institution_Name,Identification:Contact_person,Identification:Work_Area,GENERAL_INFORMATION:Secondary_movement,GENERAL_INFORMATION:Relationship,GENERAL_INFORMATION:Movement_outside,GENERAL_INFORMATION:Distance,Population:Women:Infants,Population:Women:Childs,Population:Women:Adults,Population:Women:Olders,Population:Men:Infants_001,Population:Men:Childs_001,Population:Men:Adults_001,Population:Men:Olders_001,Fuel_Cost:Main_Fuel,Fuel_Cost:Fuel_Cost_Diesel,Fuel_Cost:Fuel_Cost_Kerosene,Fuel_Cost:Fuel_Cost_Ethanol,Fuel_Cost:Fuel_Cost_Gas_cylder,ENERGY:Electricity_network,ENERGY:Covered_services,ENERGY:Power_failure,ENERGY:Demand_Curve,ENERGY:Street_Light,Urban_Planning_001:Urban_Planning,Urban_Planning_001:Period,Urban_Planning_001:Growth_area,Urban_Planning_001:Forecast,Urban_Planning_001:Land_Managment,Urban_Planning_001:Risk_Managment,Urban_Planning_001:Risk_Identified,Shelter:Plot_Area,Shelter:Plot_Limits,Shelter:Housing_Improvement,Shelter:Total_shelter,Shelter:Cost,Shelter:Shelter_Construction,Shelter:Vulnerable_Area:Vunerable_Area,Enviormental_Issues:Risk:Risk_Flood,Enviormental_Issues:Risk:Risk_dryness,Enviormental_Issues:Risk:Risk_earthquake,Enviormental_Issues:Risk:Risk_storm,Enviormental_Issues:Risk:Risk_plage,Enviormental_Issues:Deforestation,Enviormental_Issues:High_enviormental_value,Enviormental_Issues:Native_Plant,Enviormental_Issues:Native_Crops,Water_table,Sanitation:Personal_hygiene,Sanitation:Excreta,Sanitation:Open_defecation,Sanitation:Type_of_Latrine,Sanitation:Individual_Latrines,Waste_Managment:Waste_Collection,Waste_Managment:Landfill_Details:Location_1,Waste_Managment:Landfill_Details:Location_2,Waste_Managment:Landfill_Details:Adecuate_land,Water:Quality,Water:Treatment,Water:Comsuption,Water:Distribution,Water:Time,Women_Patrol,Education_Issues,Data_Access,Antenna,meta:instanceID


In [7]:
LocalLeaders.shape

(0, 22)

In [8]:
HouseHold.shape

(1, 42)

In [9]:
WomenGroup.shape

(0, 27)

In [10]:
SanitationInfra.shape

(3, 14)

In [11]:
Priorities.shape

(1, 88)

In [12]:
GeneralForm.shape

(0, 19)

In [13]:
PublicSpace.shape

(0, 13)

In [14]:
WaterInf.shape

(0, 12)

In [15]:
SanitationInf.shape

(3, 14)

In [16]:
WasteManagementInf.shape

(0, 10)

In [17]:
EnergyINF.shape

(1, 38)

In [18]:
Business.shape

(12, 25)

In [19]:
MobilityINF.shape

(1, 10)

In [52]:
ComunalServices.shape

(17, 63)

In [21]:
GeneralCitizen.shape

(0, 40)

In [22]:
Shelter.shape

(20, 27)

In [23]:
FarmyardCrop.shape

(11, 18)

In [33]:
import nltk
nltk.download("popular") # required to download the stopwords lists

from nltk.corpus import stopwords

spanish_stopwords = stopwords.words('spanish')
english_stopwords = stopwords.words('english')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [38]:
corpus = ["Pino carrasco, Roble continental, roble mediterraneo, la Haya, Eucalipto"]
X = count_vectorizer.fit_transform(corpus)
print(X.toarray())
print(count_vectorizer.get_feature_names())

[[1 1 1 1 1 2]]
['carrasco', 'continental', 'eucalipto', 'mediterraneo', 'pino', 'roble']


In [53]:
S_NoEducationCause = dfFix(Entities,"Education_Issues","Data_Access")
S_NoEducationCause = separateValues(S_NoEducationCause)
S_NoEducationCause(S_NoEducationCause,"S_NoEducationCause.csv")

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [49]:
S_DataAccess

,Data_Access


In [26]:
print(count_vectorizer.get_feature_names())

['art', 'chemistry', 'computer_scien', 'geography', 'language', 'mathematic', 'natural_scienc', 'physic', 'social_science', 'sport']


In [27]:
corpus

NameError: name 'corpus' is not defined

In [ ]:
df1.dropna()